In [1]:
import tensorflow as tf
import numpy as np
import time
import cv2
import os

2023-12-20 23:11:08.609680: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-20 23:11:08.629522: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 23:11:08.629541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 23:11:08.630222: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 23:11:08.633747: I tensorflow/core/platform/cpu_feature_guar

In [2]:
### Tensorflow

In [3]:
const255tf = tf.constant([255.0])
const128tf = tf.constant([128.0])


2023-12-20 23:11:09.198336: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 23:11:09.217494: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 23:11:09.217581: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
### Nested

In [5]:
def darken_blend_8_tf(base, active):
  return tf.where(tf.greater(base, active), active, base)

In [6]:
def color_burn_8_tf(base, active):
  return tf.where(tf.equal(active, 0), const255tf, const255tf - (const255tf - base) / active)

In [7]:
def lighten_blend_8_tf(base, active):
  return tf.where(tf.less(base, active), active, base)

In [8]:
def color_dodge_8_tf(base, active):
  return tf.where(tf.equal(active, const255tf), const255tf, base / (const255tf - active))

In [9]:
def overlay_blend_8_tf(base, active):
  return tf.where(tf.greater_equal(base, const128tf), 2 * base + base - 2 * base * base / const255tf - const128tf, 2 * base * base / const128tf)

In [10]:
def multiply_blend_8_tf(base, active):
  return base * active / const255tf

In [11]:
def linear_burn_8_tf(base, active):
  return base + active - const255tf

In [12]:
def screen_blend_8_tf(base, active):
  return base + active - base * active / const255tf

In [13]:
def linear_dodge_8_tf(base, active):
  return base + active

In [14]:
### Single

In [15]:
def normal_blend_f_tf(base, active, opacity):
  return opacity * active + (1-opacity) * base

In [16]:
def normal_blend_8_tf(base, active, opacity):
  return opacity * active + (const255tf - opacity) * base

In [17]:
### Runner

In [18]:
size = 512
np.random.seed(1)

list1 = np.random.rand(size).astype('float32')
list2 = np.random.rand(size).astype('float32')
list3 = np.random.rand(size)
list1 = tf.convert_to_tensor(list1, np.float32)
list2 = tf.convert_to_tensor(list2, np.float32)

int1 = tf.convert_to_tensor(np.random.rand(1).astype('float32'), np.float32)

In [19]:
folder = "./data/"

img_files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

bases = []
actives = []

for _file in img_files:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    rnd = np.random.rand(*img.shape).astype('float32')

    bases.append(img)
    actives.append(rnd)

In [20]:
def runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        with tf.device('/GPU:0'):
            b = tf.convert_to_tensor(bases[i], np.float32)
            a = tf.convert_to_tensor(actives[i], np.float32)
            assert a.device.endswith('GPU:0')
            assert b.device.endswith('GPU:0')
            start_time = time.perf_counter()
            f(b, a)
            end_time = time.perf_counter()
            del a
            del b
            
        total_time += (end_time - start_time) * 1000
    return total_time

def mat_timer(bases, actives, f):
    runs = 5
    times = []
    for _ in range(runs):
        times.append(runner(bases, actives, f))
    times = np.array(times)
    print(f"{np.average(times)}ms +/- {np.std(times)}ms")

In [21]:
mat_timer(bases, actives, darken_blend_8_tf)

2023-12-20 23:11:22.700491: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


339.5889590959996ms +/- 116.59198284177711ms


In [22]:
mat_timer(bases, actives, color_burn_8_tf)

721.0472451057285ms +/- 5.465780765103604ms


In [23]:
mat_timer(bases, actives, lighten_blend_8_tf) 

277.0289354491979ms +/- 2.6017532068284304ms


In [24]:
mat_timer(bases, actives, color_dodge_8_tf) 

608.13023801893ms +/- 1.5751649904720748ms


In [25]:
mat_timer(bases, actives, overlay_blend_8_tf) 

1686.642973171547ms +/- 5.304261381480963ms


In [26]:
mat_timer(bases, actives, multiply_blend_8_tf) 

366.8182907626033ms +/- 1.3902134537036872ms


In [27]:
mat_timer(bases, actives, linear_burn_8_tf) 

307.1173360571265ms +/- 0.632922966027208ms


In [28]:
mat_timer(bases, actives, screen_blend_8_tf) 

598.8244122359902ms +/- 3.224496217022755ms


In [29]:
mat_timer(bases, actives, linear_dodge_8_tf) 

188.26800496317446ms +/- 1.5647762087323793ms


In [30]:
with tf.device('/gpu'):
  %timeit list3 = normal_blend_f_tf(list1, list2, int1)

43.9 µs ± 64.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [31]:
with tf.device('/gpu'):
  %timeit list3 = normal_blend_8_tf(list1, list2, int1)

38 µs ± 41.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
